In [ ]:
!pip install llama-index
!pip install openai
!pip install langchain

In [1]:
import os
os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"

In [47]:
from langchain.llms import OpenAI

In [ ]:
# llm = OpenAI(temperature=0.9)

In [ ]:
# text = "What would be a good company name for a company that makes colorful socks?"
# print(llm(text))



Fancy Feet Socks Co.


# llamaIndex

In [48]:
from llama_index import download_loader, GPTVectorStoreIndex, ServiceContext, StorageContext
from llama_index.indices.struct_store import GPTPandasIndex
from pathlib import Path
import pandas as pd

In [ ]:
!mkdir data

mkdir: cannot create directory ‘data’: File exists


In [ ]:
# !pip show rich
!pip uninstall rich

Found existing installation: rich 13.0.1
Uninstalling rich-13.0.1:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/rich-13.0.1.dist-info/*
    /usr/local/lib/python3.10/dist-packages/rich/*
Proceed (Y/n)? y
  Successfully uninstalled rich-13.0.1


In [ ]:
!pip install rich==13.0.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached rich-13.0.1-py3-none-any.whl (238 kB)


In [ ]:
!mkdir data/new-delhi data/shanghai data/london

In [ ]:
!pwd

/content


In [ ]:
!pip install pytesseract

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [49]:
cities = ["new-delhi","london","shanghai"]
dfs = []
for city in cities:
    reviews = os.listdir(f'./data/{city}')
    city_df = []
    for file in reviews:
        if file.endswith('.txt'):
            directory_path = f'./data/{city}/{file}'
            hotel_name = ''
            if city == "new-delhi":
                hotel_name = file.split('india_new delhi_')[-1]
            elif city == "london":
                hotel_name = file.split('uk_england_london_')[-1]
            else:
                hotel_name = file.split('china_shanghai_')[-1]
            
            hotel_name = hotel_name[:-4] # removing .txt
            hotel_name = ' '.join([word.capitalize() for word in hotel_name.split('_')])

            df = pd.read_csv(directory_path , sep='\t', header=None, index_col=False, names=['Date', 'Review Heading', 'Review'])
            df['City'] = city
            df['Hotel Name'] = hotel_name
            city_df.append(df)
    final_df = pd.concat(city_df, ignore_index=True)
    final_df_1 = final_df.dropna(how='any')
    dfs.append(final_df_1)

In [50]:
dfs[0]

,Date,Review Heading,Review,City,Hotel Name
1,Nov 24 2009,Wonderful service clean rooms and well connect...,This place is perfect if you are interested in...,new-delhi,Hotel Pearl Plaza
2,Nov 24 2009,Great hotel - Clean,"My boyfriend, two friends, and I stayed at thi...",new-delhi,Hotel Pearl Plaza
3,Nov 15 2009,Nice place reality of India will hit you in th...,We stayed at this hotel a few times. The rooms...,new-delhi,Hotel Pearl Plaza
4,Jun 6 2009,Clean hotel reasonable prices,"We stayed at Pearl Plaza in Paharganj, Delhi f...",new-delhi,Hotel Pearl Plaza
7,Nov 24 2009,A Home in Karol BaghDelhi away from your Home,Hi! I am a business person from Bangalore and ...,new-delhi,Hotel C Park Inn
...,...,...,...,...,...
700,Apr 28 2006,RUN from this place!,"Terrible location. Very dirty. Broken toilet, ...",new-delhi,Hotel Chand Palace
701,Mar 3 2006,One of the best hotel Loved It,I stayed three days in this hotel and found th...,new-delhi,Hotel Chand Palace
702,Dec 4 2005,Hotel Chand Palace-poor,"Having already booked with Expedia, I was expe...",new-delhi,Hotel Chand Palace
703,Oct 22 2005,dirty building site,This is not a nice hotel. It is basic but not ...,new-delhi,Hotel Chand Palace


In [ ]:
# PandasCSVReader = download_loader("PandasCSVReader")

# loader = PandasCSVReader()
# documents = loader.load_data(file=Path('./transactions.csv'))

In [ ]:
# cities = ["new-delhi","london","shanghai"]

# UnstructuredReader = download_loader("UnstructuredReader", refresh_cache=True)

# loader = UnstructuredReader()
# doc_set = {}
# all_docs = []
# for city in cities:
#   docs = os.listdir(f'./data/{city}')
#   print(docs[0])
#   city_docs = [loader.load_data(file=Path(f'./data/{city}/{file}'), split_documents=False) for file in docs]
#   print(city_docs)
#   break
#   # insert year metadata into each city hotel 
#   for d in city_docs:
#       d.extra_info = {"city": city}
#   doc_set[city] = city_docs
#   all_docs.extend(city_docs)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


india_new delhi_hotel_pearl_plaza.txt
[[Document(text="Very friendly staff; Convenient Location Stayed here with friends as a hub in and out of New Delhi. The hotel arranged car service to/from the int'l airport and to/from the Taj Mahal the next morning. I definitely recommend this hotel as a stress-free way of traveling around and into/out of Delhi - they took care of everything and were happy to assist. Nov 24 2009 Wonderful service clean rooms and well connected with local drivers This place is perfect if you are interested in keeping the costs down, while also enjoying a clean and safe room. The frinedly and pleasant staff fulfilled all our requests, and they arranged drivers, and guides who took us to the Taj M. as well as to and from both the airport and the train station. The wokers at the hotel even gave us early morning warm Nescafe mocha drinks. If I ever make it back to Delhi I would for sure stay at the Hotel Pearl Plaza. Nov 24 2009 Great hotel - Clean My boyfriend, two f

In [51]:
index_set = {}
service_context = ServiceContext.from_defaults(chunk_size_limit=512)
for df, city in zip(dfs, cities):
    storage_context = StorageContext.from_defaults()
    # cur_index = GPTVectorStoreIndex.from_documents(
    #     doc_set[city], 
    #     service_context=service_context,
    #     storage_context=storage_context,
    # )
    cur_index = GPTPandasIndex(df=df)
    index_set[city] = cur_index
    storage_context.persist(persist_dir=f'./storage/{city}')

In [37]:
# !tar chvfz storage.tar.gz ./storage

./storage/
./storage/london/
./storage/london/docstore.json
./storage/london/vector_store.json
./storage/london/index_store.json
./storage/shanghai/
./storage/shanghai/docstore.json
./storage/shanghai/vector_store.json
./storage/shanghai/index_store.json
./storage/new-delhi/
./storage/new-delhi/docstore.json
./storage/new-delhi/vector_store.json
./storage/new-delhi/index_store.json


In [ ]:
# Load indices from disk
'''
index_set = {}
for year in years:
    storage_context = StorageContext.from_defaults(persist_dir=f'./storage/{year}')
    cur_index = load_index_from_storage(storage_context=storage_context)
    index_set[year] = cur_index
'''

# langchain

In [52]:
# do imports
from langchain.agents import Tool
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent

from llama_index.langchain_helpers.agents import LlamaToolkit, create_llama_chat_agent, IndexToolConfig

In [ ]:
# # define a decompose transform
# from llama_index.indices.query.query_transform.base import DecomposeQueryTransform
# decompose_transform = DecomposeQueryTransform(
#     llm_predictor, verbose=True
# )

# # define custom retrievers
# from llama_index.query_engine.transform_query_engine import TransformQueryEngine

# custom_query_engines = {}
# for index in index_set.values():
#     query_engine = index.as_query_engine()
#     query_engine = TransformQueryEngine(
#         query_engine,
#         query_transform=decompose_transform,
#         transform_extra_info={'index_summary': index.index_struct.summary},
#     )
#     custom_query_engines[index.index_id] = query_engine
# custom_query_engines[graph.root_id] = graph.root_index.as_query_engine(
#     response_mode='tree_summarize',
#     verbose=True,
# )

# # tool config
# graph_config = IndexToolConfig(
#     query_engine=query_engine,
#     name=f"Graph Index",
#     description="useful for when you want to answer queries that require analyzing multiple SEC 10-K documents for Uber.",
#     tool_kwargs={"return_direct": True}
# )

In [53]:
# define toolkit
index_configs = []
for y in ["new-delhi","london","shanghai"]:
    query_engine = index_set[y].as_query_engine(
        similarity_top_k=3,
    )
    tool_config = IndexToolConfig(
        query_engine=query_engine, 
        name=f"Vector Index {y}",
        description=f"useful for when you want to answer queries about the {city} hotels",
        tool_kwargs={"return_direct": True}
    )
    index_configs.append(tool_config)

In [54]:
toolkit = LlamaToolkit(
    # index_configs=index_configs + [graph_config],
    index_configs=index_configs,
)

In [55]:
memory = ConversationBufferMemory(memory_key="chat_history")
llm=OpenAI(temperature=0)
agent_chain = create_llama_chat_agent(
    toolkit,
    llm,
    memory=memory,
    verbose=True
)

In [42]:
agent_chain.run(input="hi, i am bob")



> Entering new AgentExecutor chain...

Thought: Do I need to use a tool? No
AI: Hi Bob, nice to meet you! How can I help you today?

> Finished chain.


'Hi Bob, nice to meet you! How can I help you today?'

In [43]:
agent_chain.run(input="can you help me find hotels in new delhi")



> Entering new AgentExecutor chain...

Thought: Do I need to use a tool? Yes
Action: Vector Index new-delhi
Action Input: hotels in new delhi
Observation: ['Hotel Pearl Plaza' 'Hotel C Park Inn' 'Hotel Mohan International'
 'Hotel Aster Inn' 'Hotel Metro Heights' 'Hotel Omni' 'Hotel Broadway'
 'Hotel Clark International' 'Hotel Namaskar' 'Ashok Hotel'
 'Hotel City Park' 'Hotel Apra Inn' 'Hotel Clark Heights'
 'Hotel Palace Heights' 'Hotel Mehar Castle' 'Ginger New Delhi'
 'Hotel Classic' 'Hotel Balaji Deluxe' 'The Lalit New Delhi'
 'Hotel Chand Palace']

> Finished chain.


"['Hotel Pearl Plaza' 'Hotel C Park Inn' 'Hotel Mohan International'\n 'Hotel Aster Inn' 'Hotel Metro Heights' 'Hotel Omni' 'Hotel Broadway'\n 'Hotel Clark International' 'Hotel Namaskar' 'Ashok Hotel'\n 'Hotel City Park' 'Hotel Apra Inn' 'Hotel Clark Heights'\n 'Hotel Palace Heights' 'Hotel Mehar Castle' 'Ginger New Delhi'\n 'Hotel Classic' 'Hotel Balaji Deluxe' 'The Lalit New Delhi'\n 'Hotel Chand Palace']"

In [ ]:
while True:
    text_input = input("User: ")
    response = agent_chain.run(input=text_input)
    print(f'Agent: {response}')

User: hi


> Entering new AgentExecutor chain...

Thought: Do I need to use a tool? No
AI: Hi there! How can I help you?

> Finished chain.
Agent: Hi there! How can I help you?
User: can you tell me a good hotel in new delhi


> Entering new AgentExecutor chain...

Thought: Do I need to use a tool? Yes
Action: Vector Index new-delhi
Action Input: hotel
Observation:              Date                                     Review Heading  \
1    Nov 24 2009   Wonderful service clean rooms and well connect...   
2    Nov 24 2009                                Great hotel - Clean    
3    Nov 15 2009   Nice place reality of India will hit you in th...   
4     Jun 6 2009                       Clean hotel reasonable prices   
7    Nov 24 2009       A Home in Karol BaghDelhi away from your Home   
..            ...                                                ...   
700   Apr 28 2006                               RUN from this place!   
701    Mar 3 2006                     One of the best ho